In [16]:
# Installs
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
# Google drive setup
from google.colab import drive
from os.path import join
from os import chdir

MOUNT_PATH_DRIVE = '/content/drive'
BASE_PATH = join(
      MOUNT_PATH_DRIVE, 
      "MyDrive/School/2022-2023/Sem1/ai_indistrial_perspectives/barco/barco_skin_lesion_classification"
  )
CODE_PATH = join(BASE_PATH, 'src/')

# Mount the google drive
drive.mount(MOUNT_PATH_DRIVE)

# Set the base path of the project
chdir(CODE_PATH)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Imports
# Utils
import matplotlib as plt
import numpy as np
import wandb

# DL libraries
import torch
import torch.optim as optim
from torch import nn
from torch.utils.data import DataLoader

# User libraries
from datasets.segmentationdataset import SegmentationDataset
from models.unet_model import UNet
from trainers.segmentation_model_trainer import train_segmentation_model
from util import config

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 8.0 MB/s 
     |████████████████████████████████| 182 kB 57.9 MB/s 
     |████████████████████████████████| 166 kB 64.0 MB/s 
     |████████████████████████████████| 63 kB 1.2 MB/s 
     |████████████████████████████████| 166 kB 54.3 MB/s 
     |████████████████████████████████| 162 kB 54.1 MB/s 
     |████████████████████████████████| 162 kB 45.8 MB/s 
     |████████████████████████████████| 158 kB 49.5 MB/s 
     |████████████████████████████████| 157 kB 40.6 MB/s 
     |████████████████████████████████| 157 kB 55.4 MB/s 
     |████████████████████████████████| 157 kB 47.7 MB/s 
     |████████████████████████████████| 157 kB 57.2 MB/s 
     |████████████████████████████████| 157 kB 39.4 MB/s 
     |████████████████████████████████| 157 kB 55.3 MB/s 
     |████████████████████████████████| 157 kB 48.4 MB/s 
     |████████████████████████████

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/content/drive/MyDrive/School/2022-2023/Sem1/ai_indistrial_perspectives/barco/robbe_personal_2/src


# Data

In [ ]:
# Get the data
train_segmentation_dataset = SegmentationDataset(
    config.SEGMENTATION_DATA_PATH_TRAIN_FEATURES,
    config.SEGMENTATION_DATA_PATH_TRAIN_LABELS,
    config.SEGMENTATION_TRAIN_TRANSFORMATIONS_BOTH
    )

test_segmentation_dataset = SegmentationDataset(
    config.SEGMENTATION_DATA_PATH_TEST_FEATURES,
    config.SEGMENTATION_DATA_PATH_TEST_LABELS
    )

# Place the datasets in dataloaders
train_segmentation_dataloader = DataLoader(train_segmentation_dataset, batch_size=config.SEGMENTATION_BATCH_SIZE)
test_segmentation_dataloader = DataLoader(test_segmentation_dataset, batch_size=1)



# Setup

In [ ]:
# Get the model
model = UNet(n_channels=3, n_classes=1, bilinear=False)
model.to(config.DEVICE)

# Set the optimizer
optimizer = optim.Adam(model.parameters(), lr=config.SEGMENTATION_LR)

# Set the loss fn
criteria = nn.CrossEntropyLoss()

# Set the gradient scaler
grad_scaler = torch.cuda.amp.grad_scaler.GradScaler()


# Setup weights and biasses
#wandb.login()

# Start wandb
#wandb.init(
#    settings=wandb.Settings(start_method="fork"),
#    project="test-project", 
#    name=f"experiment_1", 
#    config={
#        "learning_rate": config.SEGMENTATION_LR,
#        "batch_size": config.SEGMENTATION_BATCH_SIZE,
#        "epochs": config.SEGMENTATION_EPOCHS,
#    }
#)

/usr/local/lib/python3.10/dist-packages/torch/cuda/amp/grad_scaler.py:118: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: robberdg. Use `wandb login --relogin` to force relogin
Thread HandlerThread:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/internal/internal_util.py", line 50, in run
    self._run()
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/internal/internal_util.py", line 101, in _run
    self._process(record)
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/internal/internal.py", line 263, in _process
    self._hm.handle(record)
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/internal/handler.p

wandb: ERROR Internal wandb error: file data was not synced


# Training

In [ ]:
for epoch in range(config.SEGMENTATION_EPOCHS):
    # Set the model in training mode
    model.train()

    # Train the model
    train_loss_this_epoch = train_segmentation_model(
        model,
        optimizer,
        criteria,
        grad_scaler,
        train_segmentation_dataloader
    )
    
    # Set the model in evaluation mode
    model.eval()

    # Validate the model
    val_loss_this_epoch, sample_image_array = validate_segmentation_model(
        model,
        criteria,
        test_segmentation_dataloader,
        test_segmentation_dataset
    )

    # convert the image array into an image
    sample_image = wandb.Image(sample_image_array, caption="Top: Output, Bottom: Input")

    # Log the train loss this epoch
    wandb.log({
        'train_loss': train_loss_this_epoch/len(train_segmentation_dataloader.dataset),
        'val_loss': val_loss_this_epoch/len(test_segmentation_dataloader.dataset),
        'sample_image': sample_image
    })


# Mark the run as finished
wandb.finish()




    

Error in callback <function _WandbInit._resume_backend at 0x7fac5cdb3250> (for pre_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

  0%|          | 0/312 [00:00<?, ?it/s]


TypeError: Unexpected type <class 'numpy.ndarray'>

Error in callback <function _WandbInit._pause_backend at 0x7fac5cdb3400> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe